## A simple implementation of Naive Bayes Classifer

In [1]:
import numpy as np
import pickle
from pprint import pprint
import time
from sklearn import metrics
import matplotlib.pyplot as plt

### Loading of serialized dataset using pickle
https://docs.python.org/3/library/pickle.html 

In [2]:
def load_pkl(fname):
    """
    Reads a pkl file and deserializes a Python object into a dictionary.
    For more information refer to https://docs.python.org/3/library/pickle.html
    

    Parameters
    ----------
    fname : the name of the .pkl file

    Returns
    -------
    data :
        Returns the .pkl file as a 'dict'
    """

    # Python 3
    with open(fname, 'rb') as f:
        data = pickle.load(f, encoding='latin1')

    return data

## Exploring the dataset features

In [3]:
# to test the dataset loading
dataset = load_pkl("newsgroups.pkl")
pprint(dataset)
print(type(dataset))

{'X': <8121x100 sparse matrix of type '<class 'numpy.uint8'>'
	with 32782 stored elements in Compressed Sparse Column format>,
 'Xvalidate': <8121x100 sparse matrix of type '<class 'numpy.uint8'>'
	with 32669 stored elements in Compressed Sparse Column format>,
 'groupnames': array(['comp.*', 'rec.*', 'sci.*', 'talk.*'], dtype=object),
 'wordlist': array(['aids', 'baseball', 'bible', 'bmw', 'cancer', 'car', 'card',
       'case', 'children', 'christian', 'computer', 'course', 'data',
       'dealer', 'disease', 'disk', 'display', 'doctor', 'dos', 'drive',
       'driver', 'earth', 'email', 'engine', 'evidence', 'fact', 'fans',
       'files', 'food', 'format', 'ftp', 'games', 'god', 'government',
       'graphics', 'gun', 'health', 'help', 'hit', 'hockey', 'honda',
       'human', 'image', 'insurance', 'israel', 'jesus', 'jews', 'launch',
       'law', 'league', 'lunar', 'mac', 'mars', 'medicine', 'memory',
       'mission', 'moon', 'msg', 'nasa', 'nhl', 'number', 'oil', 'orbit',
     

In [4]:
print('Feature Dictionary, length = ', len(dataset['wordlist']), ':\n', dataset['wordlist'])
print('\n')
print('Category Labels :\n', dataset['groupnames'])

Feature Dictionary, length =  100 :
 ['aids' 'baseball' 'bible' 'bmw' 'cancer' 'car' 'card' 'case' 'children'
 'christian' 'computer' 'course' 'data' 'dealer' 'disease' 'disk'
 'display' 'doctor' 'dos' 'drive' 'driver' 'earth' 'email' 'engine'
 'evidence' 'fact' 'fans' 'files' 'food' 'format' 'ftp' 'games' 'god'
 'government' 'graphics' 'gun' 'health' 'help' 'hit' 'hockey' 'honda'
 'human' 'image' 'insurance' 'israel' 'jesus' 'jews' 'launch' 'law'
 'league' 'lunar' 'mac' 'mars' 'medicine' 'memory' 'mission' 'moon' 'msg'
 'nasa' 'nhl' 'number' 'oil' 'orbit' 'patients' 'pc' 'phone' 'players'
 'power' 'president' 'problem' 'program' 'puck' 'question' 'religion'
 'research' 'rights' 'satellite' 'science' 'scsi' 'season' 'server'
 'shuttle' 'software' 'solar' 'space' 'state' 'studies' 'system' 'team'
 'technology' 'university' 'version' 'video' 'vitamin' 'war' 'water' 'win'
 'windows' 'won' 'world']


Category Labels :
 ['comp.*' 'rec.*' 'sci.*' 'talk.*']


### Composition of the dataset 

#### 1. groupnames: The names of four newsgroups or categories: 1) comp.* 2) rec.* 3) sci.* 4) talk.*. 

In [5]:
print(dataset["groupnames"]) # The labels

['comp.*' 'rec.*' 'sci.*' 'talk.*']


#### 2. wordlist: A list of 100 words that are used as a dictionary or feature dimensions to categorise the posts from these newsgroups. 

In [6]:
print(dataset["wordlist"][:15]) # The 100 words 

['aids' 'baseball' 'bible' 'bmw' 'cancer' 'car' 'card' 'case' 'children'
 'christian' 'computer' 'course' 'data' 'dealer' 'disease']


#### 3. X: A binary matrix. Each row corresponds to a post, and each column corresponds to a word from the word list. A value of 1 means that the word occured in the post. 

In [7]:
print(dataset["X"][0])

  (0, 51)	1


#### 4. y: A vector with values 1 through 4, with the value corresponding to the newsgroup that the post came from. 

In [21]:
print(dataset["y"])

[1 4 4 ... 2 4 4]


#### 5. Xvalidate and yvalidate: consists of test data from news group posts with labels to test the accuracy of the classiﬁer. 

In [9]:
print(dataset["Xvalidate"][0])
print(dataset["yvalidate"][:5])

  (0, 5)	1
  (0, 11)	1
  (0, 26)	1
  (0, 33)	1
  (0, 35)	1
  (0, 38)	1
  (0, 67)	1
  (0, 69)	1
  (0, 75)	1
  (0, 85)	1
  (0, 99)	1
[4 1 3 4 1]


## Testing Assumptions

In [23]:
M = np.zeros(400).reshape(4,100)
X = dataset["X"]
X = X.toarray()
y = dataset["y"]
N,D = X.shape
for i in range(0,N):
    for j in range(0,D):
        if y[i] == 1:
            M[0][j] += X[i][j]
        if y[i] == 2:
            M[1][j] += X[i][j]
        if y[i] == 3:
            M[2][j] += X[i][j]
        if y[i] == 4:
            M[3][j] += X[i][j]
# STEP 3: LA PLACE +1 SMOOTHENING
for i in range(0,4):
    for j in range(0,D):
        if M[i][j] == 0:
            M[i][j] += 1
M

array([[  4.,   1.,   1.,   1.,   3.,  11., 291., 140.,  13.,  11., 323.,
        109., 218.,  25.,   1., 216., 167.,   3., 252., 290., 144.,  19.,
        513.,   7.,  12.,  93.,   8., 253.,   2., 131., 169.,  28.,  18.,
         16., 250.,   7.,   1., 520.,  20.,   1.,   1.,  30., 154.,   2.,
          4.,   1.,   1.,   6.,   9.,   2.,   1., 246.,   3.,   9., 205.,
          7.,   3.,   2.,  62.,   1., 205.,   1.,   2.,   1., 251., 171.,
          2., 102.,  10., 479., 342.,   1., 221.,   1.,  97.,  10.,   5.,
         81., 125.,   1., 134.,   4., 337.,   2.,  90.,  81.,  10., 422.,
          4.,  99., 283., 275., 188.,   1.,   3.,   8., 127., 559.,   2.,
        110.],
       [  1., 207.,   1., 105.,   3., 309.,  18., 105.,  20.,   9.,  59.,
        153.,  30.,  53.,   3.,  12.,  12.,   6.,   1., 109.,  43.,  18.,
        159., 108.,  21., 126., 131.,  10.,  10.,  10.,   6., 256.,  31.,
         16.,   5.,  18.,   4., 138., 152., 186.,  67.,  26.,   7.,  54.,
          1.,   6.,   1

In [15]:
C = 4
p_xy = np.zeros((D,C,2))
for i in range(0,C): 
    for j in range(0,D):
        p_xy[j][i][1] = M[i][j]/N
        p_xy[j][i][0] = 1 - M[i][j]/N
p_xy

array([[[9.99507450e-01, 4.92550179e-04],
        [1.00000000e+00, 0.00000000e+00],
        [9.96798424e-01, 3.20157616e-03],
        [9.96552149e-01, 3.44785125e-03]],

       [[1.00000000e+00, 0.00000000e+00],
        [9.74510528e-01, 2.54894717e-02],
        [9.99384312e-01, 6.15687723e-04],
        [9.98768625e-01, 1.23137545e-03]],

       [[1.00000000e+00, 0.00000000e+00],
        [1.00000000e+00, 0.00000000e+00],
        [9.99876862e-01, 1.23137545e-04],
        [9.66260313e-01, 3.37396872e-02]],

       [[1.00000000e+00, 0.00000000e+00],
        [9.87070558e-01, 1.29294422e-02],
        [9.99261175e-01, 7.38825268e-04],
        [1.00000000e+00, 0.00000000e+00]],

       [[9.99630587e-01, 3.69412634e-04],
        [9.99630587e-01, 3.69412634e-04],
        [9.95567048e-01, 4.43295161e-03],
        [9.98522349e-01, 1.47765054e-03]],

       [[9.98645487e-01, 1.35451299e-03],
        [9.61950499e-01, 3.80495013e-02],
        [9.96182736e-01, 3.81726388e-03],
        [9.91996060e-01,

## Implement fit method for Naive Bayes learning 

In [24]:
def fit(X, y):
    """
    This function should fit the training set of features and labels to a Naive Bayes Model.
    
    """
    # Shape of the feature matrix 
    N, D = X.shape # THESE SHOULD BE 8121, 100

    # Compute the number of class labels
    C = np.unique(y).size # C = 4

    # Create a mapping from the labels to 0,1,2,...so that we can store things in numpy arrays
    labels = dict() 
    for index, label in enumerate(np.unique(y)):
        labels[index] = label # {0: 1, 1: 2, 2: 3, 3: 4}

    # Compute the probability of each class i.e p(y==c)
    counts = np.zeros(C) # array([0., 0., 0., 0.])
    
    # STEP 1: COMPUTING PRIOR PROBABILITIES
    for index, label in labels.items(): # For each type of label
        counts[index] = np.sum(y==label) # Count the label's frequencies and store them in counts
        p_y = counts / N # p_y is the probability of the label
        # counts = [2287. 1775. 1349. 2710.]
        # p_y = [0.28161556 0.21856914 0.16611255 0.33370275]
        
    # To convert a sparse array to regular array
    X = X.toarray() # SEE DIFFERENCE BETWEEN THESE TYPES OF ARRAYS
        
        
    """ ******** implement YOUR CODE HERE for conditional probablities ***** """       
    
    
    # STEP 2: COMPUTING FREQUENCY OF THERE BEING THE WORD
    M = np.zeros(400).reshape(4,100)
    for i in range(0,N):
        for j in range(0,D):
            if y[i] == 1:
                M[0][j] += X[i][j]
            if y[i] == 2:
                M[1][j] += X[i][j]
            if y[i] == 3:
                M[2][j] += X[i][j]
            if y[i] == 4:
                M[3][j] += X[i][j]
            
    # STEP 3: LA PLACE +1 SMOOTHENING
    for i in range(0,C):
        for j in range(0,D):
            if M[i][j] == 0:
                M[i][j] += 1
    
    # STEP 4: COMPUTING CONDITIONAL PROBABILITIES 
    p_xy = np.zeros((D,C,2))
    for i in range(0,C): 
        for j in range(0,D):
            p_xy[j][i][1] = M[i][j]/N
            p_xy[j][i][0] = 1 - M[i][j]/N
    
            
    """ *************************************************************** """
    # Save parameters in model as dict
    model = dict() # Model is a dictionary with four keys.
    
    model["p_y"] = p_y # Probability of labels
    model["p_xy"] = p_xy # Probability of x given y
    model["n_classes"] = C # How many classes (labels)
    model["labels"] = labels # The actual classes (labels)

    return model

### Predict method

In [25]:
def predict(model, X):
    N, D = X.shape # THESE SHOULD BE 8121, 100
    C = model["n_classes"] 
    p_xy = model["p_xy"]
    p_y = model["p_y"]
    labels = model["labels"]

    y_pred = np.zeros(N)

    for n in range(N):
        # Compute the probability for each class
        # This could be vectorized but we've tried to provide an intuitive version.
        probs = p_y.copy()

        for d in range(D):
            if X[n, d] == 1:
                for c in range(C):
                    probs[c] *= p_xy[d, c, 1]

            elif X[n, d] == 0:
                for c in range(C):
                    probs[c] *= p_xy[d, c, 0]

        y_pred[n] = labels[np.argmax(probs)]

    return y_pred

In [26]:
start = time.time()

# Load the dataset
dataset = load_pkl("newsgroups.pkl")

# Extract training data and test data
X = dataset["X"]
y = dataset["y"]

#X_valid = dataset["Xvalidate"]
#y_valid = dataset["yvalidate"]

#print(X_valid.toarray())

#print(type(X_valid))

# Test/validation set to test the accuracy of our model on data it has not seen
# Choosing only 300 test samples
X_valid = dataset["Xvalidate"][:300]
y_valid = dataset["yvalidate"][:300]

print("Model is fitting")
model = fit(X,y)

print("Model is predicting")
y_pred = predict(model, X_valid)

print(y_pred)

v_error = np.mean(y_pred != y_valid)
print("Naive Bayes Validation error: %.3f" % v_error)


measures_info = metrics.classification_report(y_valid, y_pred)

print('Performance measures: \n', measures_info)



end = time.time()

print('time taken:', end - start)

Model is fitting
Model is predicting
[4. 1. 1. 4. 1. 4. 3. 4. 3. 1. 4. 2. 4. 4. 4. 4. 1. 2. 1. 4. 4. 1. 2. 1.
 2. 2. 4. 4. 2. 4. 4. 4. 1. 1. 1. 1. 4. 1. 1. 3. 2. 3. 4. 1. 4. 1. 4. 4.
 1. 1. 1. 4. 2. 1. 3. 1. 4. 1. 1. 4. 4. 1. 2. 1. 4. 2. 1. 1. 1. 4. 4. 4.
 1. 4. 4. 4. 1. 4. 1. 3. 4. 2. 4. 1. 4. 1. 1. 1. 2. 1. 4. 1. 1. 1. 1. 1.
 1. 4. 2. 1. 4. 1. 2. 1. 1. 4. 1. 4. 2. 1. 1. 1. 1. 4. 4. 1. 1. 1. 4. 2.
 4. 4. 2. 4. 4. 1. 4. 1. 4. 3. 3. 1. 3. 4. 3. 1. 4. 4. 1. 3. 3. 4. 4. 4.
 4. 1. 1. 4. 2. 3. 4. 4. 4. 1. 4. 1. 4. 2. 3. 1. 4. 3. 4. 1. 3. 4. 3. 1.
 1. 1. 4. 1. 1. 1. 4. 4. 1. 4. 2. 4. 4. 2. 3. 2. 4. 4. 1. 1. 4. 4. 4. 4.
 1. 2. 1. 1. 1. 4. 4. 1. 4. 4. 4. 4. 1. 4. 3. 4. 2. 4. 4. 2. 4. 4. 1. 2.
 3. 1. 1. 1. 2. 3. 2. 1. 4. 1. 1. 4. 4. 2. 4. 4. 4. 4. 4. 3. 4. 4. 1. 4.
 4. 1. 4. 4. 1. 1. 4. 2. 1. 2. 3. 1. 3. 1. 4. 4. 1. 4. 4. 4. 1. 4. 4. 4.
 1. 1. 4. 3. 4. 4. 4. 1. 3. 1. 4. 1. 4. 4. 2. 4. 2. 4. 1. 4. 2. 3. 4. 2.
 1. 4. 1. 1. 4. 2. 4. 1. 4. 4. 4. 3.]
Naive Bayes Validation error: 0.203
Performance m

## Conclusions 
The model is much more precise than the "dummy" model that preceded it. The Validation Error is only 20.3%, compared to 65%, which means accuracy is at 79.7%, compared to 45%. 
Altough, the computation takes much more time, and is almost five times slower: 19.74s, compared to 4.26s. Surely, there is room for improvement in terms of both accuracy and resource and time management. 
Perhaps there is another way to write the solution that doesn't involve nested ifs, such as recursion.

With an investment of just a bit more time and dedication, the functions used here can be extended in a class with more useful methods.